Import some stuff

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 12) # set default size of plots
plt.rcParams.update({'font.size': 18}) # set default font size
plt.rcParams['image.cmap']='inferno' # set color map

A function to see when we're done

In [ ]:
def am_done(v):
    '''Return true if all elements of v the same. Else false.'''
    u=np.unique(v) # unique  elements of v
    if u.shape[0]>1:
        return False
    else:
        return True

The main **simulation**. Run a process with rate $L$. Waiting times are distributed exponentially with mean $\beta = 1/L$. When an event occurs pick an individual uniform at random to reproduce and pick an individual uniform at random to be killed.

In [ ]:
### Parameter ###

L=50

#################

X=np.zeros((1,L),dtype='int') # initialize (we will use np.append to append to this)
X[0,:]=np.arange(L)

time=0.0
times_list=[] # list of times for sample path

rate=L 
beta=1.0/rate # mean wait time

while True: # loop until done 
    wait=np.random.exponential(beta) # wait time
    times_list.append((time,time+wait))
    time+=wait
    
    old=X[-1,:] # index -1 is useful. In this case it gives the last row
    new=old.copy() # make a copy
    
    type_to_spread=np.random.choice(old) 

    idx=np.random.choice(np.arange(L))
    new[idx]=type_to_spread # infect    
    new=np.sort(new)  
    
    if(am_done(new)):
        break    
        
    X=np.append(X,[new],axis=0)
    
X=np.append(X,[new],axis=0) # add this for completeness 
times_list.append((time,time+2*wait))

Plot the X matrix

In [ ]:
plt.pcolormesh(X) # Plot dynamics!
plt.title('X Matrix')
plt.xlabel('Individual')
plt.ylabel('Timestep (non-uniform!)')

Convert to the N matrix

In [ ]:
def X_to_N(X):
    N=np.zeros_like(X)
    length=np.shape(N)[0]
    L=np.shape(N)[1]
    for row in range(length):
        row_list=list(X[row,:])
        for i in range(L):
            N[row,i]=row_list.count(i)   
    return N

In [ ]:
N=X_to_N(X)

Vizualise N matrix

In [ ]:
plt.pcolormesh(N) # Plot dynamics!
plt.title('N Matrix')
plt.xlabel('Type')
plt.ylabel('Timestep (non-uniform!)')

In [ ]:
# def plot_path_true(tlst,n,col): # this is too slow unfortunately 
#     length=len(tlst)
#     for i in range(length):
#         t=tlst[i]
#         plt.plot([t[0],t[1]],[n[i],n[i]],c=col,lw=3)

Make a list of just start times. Not technically the best way to plot but this is much quicker..

In [ ]:
start_times=[]
for i in range(len(times_list)):
    start_times.append(times_list[i][0])

In [ ]:
import matplotlib
import matplotlib.cm as cm

norm = matplotlib.colors.Normalize(vmin=0, vmax=L-1) # we want the same colors as for our heatmaps above!
mapper = cm.ScalarMappable(norm=norm, cmap=cm.inferno)

for i in range(L):
    plt.plot(start_times,N[:,i],c=mapper.to_rgba(i),lw=3)
    
plt.xlabel('Time, $t$')
plt.ylabel('$N_t$')